In [ ]:
#検証時、Routeにデータを設定する際利用したソースファイル

In [1]:
import psycopg2
import datetime

In [2]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [8]:
sql = "select start_mesh_area,end_mesh_area,population,start_time,end_time from kddi_location WHERE start_time = '16:30:00' AND yyyymmdd = 20211003"
cur2 =connection.cursor()
cur2.execute(sql)
for row in cur2:
    start_mesh_area = row[0]
    end_mesh_area = row[1]
    population = row[2]
    start_time = row[3]
    end_time = row[4]

    sql = "SELECT latitude,longitude from pointdata WHERE cast(grid_code as BIGINT) = "+str(start_mesh_area)
    cur =connection.cursor()
    cur.execute(sql)
    start_lat = 0
    for row in cur:
        start_lat = row[0]
        start_lng = row[1]
    cur.close()
    
    if start_lat == 0:
        print("start_lat not found.")
        continue

    sql = "SELECT latitude,longitude from pointdata WHERE cast(grid_code as BIGINT) =  "+str(end_mesh_area)
    cur =connection.cursor()
    cur.execute(sql)
    end_lat = 0
    for row in cur:
        end_lat = row[0]
        end_lng = row[1]
    cur.close()

    if end_lat == 0:
        print("end_lat not found.")
        continue

    sql = "SELECT id FROM roadmap_vertices_pgr"
    sql += " ORDER BY the_geom <-> ST_GeomFromText('POINT (" + str(start_lng) + " " + str(start_lat) + ")', 4612)"
    sql += " LIMIT 1 ;"
    cur =connection.cursor()
    cur.execute(sql)
    firstid = 0
    for row in cur:
        firstid = row[0]
        #print(row[0])
    cur.close()

    if firstid == 0:
        print("firstid not found.")
        continue
        
    sql = "SELECT id FROM roadmap_vertices_pgr"
    sql += " ORDER BY the_geom <-> ST_GeomFromText('POINT ("+str(end_lng)+" "+str(end_lat)+")', 4612)"
    sql += " LIMIT 1 ;"
    cur =connection.cursor()
    cur.execute(sql)
    for row in cur:
        lastid = row[0]
        #print(row[0])
    cur.close()

    sql = "SELECT seq, node, edge, cost FROM pgr_dijkstra("
    sql += "'SELECT osm_id AS id, source, target, ST_Length(geom) AS cost FROM roadmap',"
    sql += " "+ str(firstid) +","
    sql += " "+ str(lastid) +","
    sql += " false"
    sql += ");"
    cur =connection.cursor()
    cur.execute(sql)
    two_target_list = []
    costs = 0
    cnts = 0
    for row in cur:
        print(row)
        two_target_list.append(row[2])
        costs = costs + row[3]
        cnts = cnts + 1
    cur.close()
    
    if cnts == 0:
        continue
    
    sttime = datetime.datetime.strptime(str(start_time), '%H:%M:%S')
    wktime = sttime
    edtime = datetime.datetime.strptime(str(end_time), '%H:%M:%S')
    print(sttime)
    print(edtime)
    scale = 0
    for i in range(100):
        print(wktime)
        scale += 1
        wktime = wktime + datetime.timedelta(minutes=15)
        if wktime > edtime:
            break;

    # 時刻幅＞ルート数
    wktime = sttime
    if scale > cnts:
        for i in range(scale):
            j = round(cnts / scale * i)
            print(str(wktime.time()) + "#" + str(two_target_list[j]))
            wktime = wktime + datetime.timedelta(minutes=15)
    else if scale = cnts:
        for i in range(scale):
            print(str(wktime.time()) + "#" + str(two_target_list[i]))
            wktime = wktime + datetime.timedelta(minutes=15)
    else:
        wkj = 0
        for i in range(scale):
            j = round(scale / cnts * i)
            if wkj != j:
                print(str(wktime.time()) + "#" + str(two_target_list[i]))
                wktime = wktime + datetime.timedelta(minutes=15)
cur2.close()


start_lat not found.
start_lat not found.
start_lat not found.
start_lat not found.
end_lat not found.
(1, 295, 81157690, 0.01960902430725263)
(2, 82, 324020148, 0.0016491837302280937)
(3, 80, 315499969, 0.01485647005812323)
(4, 3161, 309968195, 0.002479178737146623)
(5, 3160, 432621272, 0.002819657311950025)
(6, 3458, 432621260, 0.0003110468376663744)
(7, 2639, 228330075, 0.004976893346644455)
(8, 2638, 228330076, 0.003656349811768765)
(9, 2514, -1, 0.0)
1900-01-01 16:30:00
1900-01-01 17:30:00
1900-01-01 16:30:00
1900-01-01 16:45:00
1900-01-01 17:00:00
1900-01-01 17:15:00
1900-01-01 17:30:00
(1, 1594, 155559747, 0.00029636727922787267)
(2, 1595, 155559765, 0.0040667964383934215)
(3, 1596, 315499980, 0.00046043550325465575)
(4, 1555, 155559671, 0.0012906688918424935)
(5, 1568, 294498695, 0.0024304771019953747)
(6, 2956, 294498691, 0.00041600792078991814)
(7, 2955, 324020143, 0.0001485451830010425)
(8, 81, 324020147, 0.014989603645245134)
(9, 3238, 315499970, 0.0024805808587580755)
(10,

IndexError: list index out of range

In [12]:
keylist = []
for item in two_target_list:
    sql = "SELECT a.grid_code"
    sql += " FROM pointdata a, roadmap r"
    sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.003"
    sql += " order by ST_Distance(a.geom, r.geom);"
    cur =connection.cursor()
    cur.execute(sql)
    childlist = []
    for row in cur:
        childlist.append(row[0])
        print(row[0])
    worklist = [item, childlist]
    keylist.append(worklist)
    cur.close()

In [95]:
keylist

[[297130200,
  ['51320624211',
   '51320614433',
   '51320624214',
   '51320624213',
   '51320624212',
   '51320614431',
   '51320614344',
   '51320624122',
   '51320614342',
   '51320624232',
   '51320624231',
   '51320624124',
   '51320614434',
   '51320614413',
   '51320614432',
   '51320624223',
   '51320614324',
   '51320624221',
   '51320624234',
   '51320624142',
   '51320624233',
   '51320624241',
   '51320614343',
   '51320624121',
   '51320614443',
   '51320614414',
   '51320614341']],
 [1025576099,
  ['51320614143',
   '51320614322',
   '51320614324',
   '51320614433',
   '51320614132',
   '51320614431',
   '51320614321',
   '51320614342',
   '51320614134',
   '51320614141',
   '51320614144',
   '51320614413',
   '51320614344',
   '51320624211',
   '51320614114',
   '51320614411',
   '51320624122',
   '51320614323',
   '51320614312',
   '51320614123',
   '51320614142',
   '51320614131',
   '51320614434',
   '51320614233',
   '51320614133',
   '51320624213',
   '51320614432',

In [96]:
connection.close()

In [97]:
#Kepler.glの導入
from keplergl import KeplerGl
#pandasの導入
import pandas as pd
import numpy as np

In [98]:
# 交通量データの読込み
csv_file = r'./../data/example/Sdata.csv'
df = pd.read_csv(csv_file, parse_dates=True, dtype={'population':float})

In [99]:
# １時間単位・メッシュコードでGroup byして、結果をagg関数にsum関数を渡して集計
totalling_df = df.groupby(['datetime', 'start_mesh_area']).agg(np.sum)
output_df = totalling_df.unstack(fill_value=0)
rows = output_df.columns.values

In [100]:
# 列名を指定
df = pd.read_csv('C:\\Users\\BIGDATA_DEV\\Desktop\\実証実験事業\\Notebook\\pred_lstm1.csv', names=rows)

In [101]:
df

population                                                              \
   50323424234 50323454332 50323454414 50323457331 50323457431 50323462233   
0     0.001719    0.000091   -0.000832   -0.001781   -0.008739    0.000225   
1     0.001388   -0.000274    0.001073   -0.001403   -0.005246    0.000159   
2     0.000544   -0.000883    0.001197   -0.000888   -0.000230   -0.000680   
3     0.000374   -0.001077    0.001594   -0.000874    0.001235   -0.000943   
4     0.000273   -0.001169    0.001876   -0.000939    0.002041   -0.001069   
5     0.000225   -0.001142    0.001970   -0.001004    0.002389   -0.001094   
6     0.000203   -0.001036    0.001918   -0.001029    0.002485   -0.001061   
7     0.000198   -0.000888    0.001773   -0.001014    0.002440   -0.000994   
8     0.000278   -0.000746    0.001676   -0.000965    0.002215   -0.000935   
9     0.000363   -0.000634    0.001607   -0.000866    0.001912   -0.000835   
10    0.000440   -0.000548    0.001554   -0.000760    0.001604   -0.000708   
11    0.000506   -0.000482    0.001515   -0.000654    0.001308   -0.000595   
12    0.000559   -0.000422    0.001471   -0.000559    0.001045   -0.000502   
13    0.000590   -0.000356    0.001406   -0.000483    0.000843   -0.000427   
14    0.000604   -0.000286    0.001327   -0.000427    0.000699   -0.000371   
15    0.000610   -0.000217    0.001244   -0.000386    0.000596   -0.000330   
16    0.000612   -0.000155    0.001169   -0.000357    0.000520   -0.000302   
17    0.000610   -0.000103    0.001102   -0.000336    0.000457   -0.000278   
18    0.000611   -0.000062    0.001050   -0.000322    0.000399   -0.000262   
19    0.000615   -0.000030    0.001012   -0.000311    0.000345   -0.000253   
20    0.000622   -0.000005    0.000984   -0.000302    0.000295   -0.000248   
21    0.000630    0.000015    0.000964   -0.000296    0.000250   -0.000247   
22    0.000639    0.000030    0.000950   -0.000290    0.000211   -0.000248   
23    0.000647    0.000041    0.000941   -0.000286    0.000178   -0.000250   

                                                    ...              \
   50323462234 50323462411 50323462412 50323462424  ... 51320614321   
0     0.000826    0.000794   -0.000172    0.000072  ...   -0.004631   
1     0.000916   -0.000241    0.001180   -0.000465  ...   -0.003109   
2     0.001931   -0.001507    0.002788   -0.001297  ...    0.002996   
3     0.002107   -0.002053    0.003509   -0.001609  ...    0.003838   
4     0.002162   -0.002359    0.003913   -0.001779  ...    0.004115   
5     0.002088   -0.002392    0.003942   -0.001790  ...    0.003735   
6     0.001940   -0.002239    0.003706   -0.001691  ...    0.003024   
7     0.001736   -0.001991    0.003311   -0.001525  ...    0.002416   
8     0.001445   -0.001811    0.002938   -0.001341  ...    0.002108   
9     0.001185   -0.001655    0.002559   -0.001153  ...    0.002394   
10    0.000968   -0.001520    0.002211   -0.000982  ...    0.002893   
11    0.000778   -0.001406    0.001895   -0.000822  ...    0.003486   
12    0.000607   -0.001299    0.001594   -0.000672  ...    0.003898   
13    0.000453   -0.001175    0.001310   -0.000538  ...    0.003870   
14    0.000314   -0.001037    0.001053   -0.000424  ...    0.003374   
15    0.000188   -0.000899    0.000833   -0.000328  ...    0.002575   
16    0.000076   -0.000775    0.000651   -0.000250  ...    0.001680   
17   -0.000017   -0.000668    0.000502   -0.000185  ...    0.000879   
18   -0.000096   -0.000585    0.000385   -0.000132  ...    0.000214   
19   -0.000164   -0.000523    0.000295   -0.000087  ...   -0.000313   
20   -0.000222   -0.000478    0.000225   -0.000051  ...   -0.000715   
21   -0.000271   -0.000445    0.000172   -0.000021  ...   -0.001018   
22   -0.000311   -0.000422    0.000131    0.000004  ...   -0.001246   
23   -0.000345   -0.000406    0.000100    0.000024  ...   -0.001420   

                                                                            \
   51320614322 51320614323 51320614324 51320614342

In [110]:
childlist = []
for keyitem in keylist:
    itemno = 0
    itemparam = 0
    for meshid in keyitem[1]:            #メッシュ分ループ
        for index,item in df.iterrows(): #データ読み込み
            i = 0 
            for rowName in output_df.columns.values: #データの列名をループ
                if str(meshid) == str(rowName[1]): #列名と一致した
                    print(str(meshid) + " " + str(rowName[1]))
                    itemno += 1
                    itemparam += item[i]
                    break
                i = i + 1
            break
    if itemno == 0:
        print(str(meshid) + "not found")
        worklist = [ keyitem[0], 0 ]
    else:
        worklist = [ keyitem[0], (itemparam / itemno) ]
    childlist.append(worklist)
         

51320614433 51320614433
51320614431 51320614431
51320614342 51320614342
51320614324 51320614324
51320614143 51320614143
51320614322 51320614322
51320614324 51320614324
51320614433 51320614433
51320614132 51320614132
51320614431 51320614431
51320614321 51320614321
51320614342 51320614342
51320614141 51320614141
51320614114 51320614114
51320614323 51320614323
51320614312 51320614312
51320614123 51320614123
51320614142 51320614142
51320614131 51320614131
51320614113 51320614113
51320614112 51320614112
51320614124 51320614124
51320614132 51320614132
51320614111 51320614111
51320614113 51320614113
51320603444 51320603444
51320614114 51320614114
51320613222 51320613222
51320604333 51320604333
51320614131 51320614131
51320614141 51320614141
51320614143 51320614143
51320603442 51320603442
51320614112 51320614112
51320613224 51320613224
51320604331 51320604331
51320614123 51320614123
51320614312 51320614312
51320614321 51320614321
51320603443 51320603443
51320613221 51320613221
51320603424 5132

In [111]:
childlist

[[297130200, -0.004263022332452237],
 [1025576099, -0.000824094638422442],
 [1025576101, 0.0014165788108419458],
 [1025576102, 0.002061894812088992],
 [-1, 0]]